In [1]:
#import mariadb
import tqdm
import pickle
import pandas as pd
import mysql.connector
from decimal import Decimal
from neo4j import GraphDatabase
from datetime import timedelta

In [2]:
df_pw = pd.read_csv("pw.csv", delimiter=",")

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import pymysql
db_url = {
    'database': 'DarknetScrape',
    'drivername': 'mysql',
    'username': df_pw["name"][0],
    'password': df_pw["password"][0],
    'host': '127.0.0.1',
    'port': 3306,
    'query': {'charset': 'utf8'},
}

db_connection = create_engine(URL(**db_url), encoding="utf8")

c:\users\d067608\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:14: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  


In [4]:
db = GraphDatabase.driver("neo4j://127.0.0.1:7687", auth=(df_pw["name"][0], df_pw["password"][0]))
session = db.session()

In [8]:
df_sqlcsv = pd.read_csv('reviews_price_btc.csv')
df_sqlcsv.time = pd.to_datetime(df_sqlcsv.time)
df_sqlcsv["price_btc_reviews_satoshi"] = df_sqlcsv["price_btc_review"] * 100000000
df_sqlcsv = df_sqlcsv.sort_values(by="time",ascending=True)

In [6]:
j_addresses=pd.read_csv('candidates_final.csv')
j_addresses=j_addresses.drop(columns=['Unnamed: 0'])
j_addresses.head(1)

,Vendor,Address
0,CNSHarmony,34Xo7fKRe9MjYBf3tyJjAFg9qX2kuFuhA1


In [9]:
query = """select a.vendor_id, b.username from (
SELECT vendor_id, count(*) as count
from vendors
group by vendor_id
) as a
JOIN (SELECT vendor_id, username from vendors where market = 'Cryptonia') b
on a.vendor_id = b.vendor_id
where a.count = 2;
"""
df = pd.read_sql(query, con=db_connection)
df=df[df['username'].isin(j_addresses['Vendor'])]
df_sqlcsv = df_sqlcsv[df_sqlcsv["vendor_id"].isin(df["vendor_id"])]
df_sqlcsv.head()

,Unnamed: 0,price_eur,time,customer,vendor_id,vendor_review_id,price_btc_review,price_btc_reviews_satoshi
26608,26608,165.14,2017-10-09 14:35:07,tk [5],a081033ab42c85ae4101549cd64ff4cb,105344,0.040876,4.087624e+06
26607,26607,401.17,2017-10-19 19:25:48,tk [5],a081033ab42c85ae4101549cd64ff4cb,105343,0.083459,8.345885e+06
25551,25551,277.69,2017-10-20 13:31:27,I***5 [2],8438ba1a66f83580db67292ecced8f9c,104261,0.056041,5.604125e+06
25550,25550,104.36,2017-10-23 22:06:34,np [3],8438ba1a66f83580db67292ecced8f9c,104260,0.020842,2.084157e+06
26606,26606,110.91,2017-12-05 23:06:20,a**c [1],a081033ab42c85ae4101549cd64ff4cb,105342,0.011203,1.120269e+06


In [18]:
neo_results_a = []
neo_query = session.run("""
    Match (vendor:Address) where vendor.address='3DdUb8Tt59tvZacikVt6R9pYWYqzGqgxff' 
with vendor
match (vendor)-[r:SENDS]->(t:Transaction)
with t
match (vendor2:Address)-[s:SENDS]->(t)
return distinct vendor2.address
    """)
    
for r in neo_query:
    neo_results_a.append(r[0])

In [23]:
# Assumptions:
# - inDegree: we don't know the inDegree, there might be multiple mixing addresses spending to the vendor
#           - if we want to reduce the amount of result it could be changed to e.g. <= X
# - outDegree: it is likely that only one vendor address receives the money, it might be that money is going back as change
#           - outDegree = 2: change is going back to the market
#           - outDegree = 1: exact amount is transferred to the vendor without change
#           - outDegree > 2: A) several vendor addresses of the same vendor wallet are paid at the same time 
#                            B) several vendors are paid at the same time
# (t)-[r1:RECEIVES]->(c:Address)-[s:SENDS]->(t) filter for change going back
transaction_t2 = pd.DataFrame()
for i in df_sqlcsv.itertuples():
    neo_results = []
    neo_results2 = []

    end = str(i.time.date())
    minSum = str(round(i.price_btc_reviews_satoshi,0))
    maxSum = str(round(i.price_btc_reviews_satoshi*1.40,0))
    addresses = str(neo_results_a)
    
    neo_query = session.run("""
    MATCH (vendor:Address)
        WHERE vendor.address IN """ + addresses + """
    WITH vendor, '""" + end + """' as endDate, """
    + minSum + """ as minSum, """ + maxSum +""" as maxSum
    MATCH (t:Transaction)-[r:RECEIVES]->(vendor)
        WHERE   ( t.outDegree = 2 OR t.outDegree = 1 ) AND t.date = date(endDate) 
        and minSum <= t.inSum <=maxSum
    WITH t, minSum, endDate
    MATCH (t)-[r:RECEIVES]->(a:Address)
        WHERE minSum  <= r.value <= minSum * 1.02 
    RETURN t, r.value
    """)
    
    for r in neo_query:
        neo_results.append(r[0])
        neo_results2.append(r[1])

    
    df_neo4j = pd.DataFrame(neo_results)
    df_neo4j["vendor"] = i.vendor_id
    df_neo4j["review"] = i.vendor_review_id
    df_neo4j["rvalue"] = pd.Series(neo_results2)
    transaction_t2 = pd.concat([transaction_t2,df_neo4j])
    
display(transaction_t2.head() )

c:\users\d067608\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,vendor,review,rvalue,date,inDegree,inSum,outDegree,outSum,txid
0,fcd6278f5d88fc20e3eab0db9ab0403c,110079,234000.0,2019-04-28,1.0,256234.0,1.0,234000.0,bcb6ae9e5b1120990eb64e638dd92992e6598ecb982b9e...
0,8438ba1a66f83580db67292ecced8f9c,104255,5634030.0,2019-04-29,1.0,7510896.0,2.0,7510218.0,b4b20cff233501e1ad81a440a890dfe2ec91f5b874887e...
0,abfc3a65538a6ec86502b2b498b6b4a6,105959,1822271.0,2019-05-04,1.0,1837565.0,1.0,1822271.0,c9a130937847d6876252545eb7382a0c83ea834e7ad284...
0,cd41d7e54101b581036b59bb06cd1548,107532,4400000.0,2019-05-05,2.0,4958488.0,2.0,4956992.0,ddfb1adc23fb4cf644d9363b15185888443df22916c3b4...
0,abfc3a65538a6ec86502b2b498b6b4a6,105954,15611288.0,2019-05-06,1.0,15730847.0,2.0,15727082.0,d269a60ddef703cbb789c3bc187b5a339beaf2db303ac1...


In [27]:
transaction_t2.groupby(by="review").count()

,vendor,rvalue,date,inDegree,inSum,outDegree,outSum,txid
review,,,,,,,,
100612,1,1,1,1,1,1,1,1
100614,2,2,2,2,2,2,2,2
100615,2,2,2,2,2,2,2,2
100617,1,1,1,1,1,1,1,1
100618,83,83,83,83,83,83,83,83
...,...,...,...,...,...,...,...,...
110035,1,1,1,1,1,1,1,1
110053,1,1,1,1,1,1,1,1
110058,1,1,1,1,1,1,1,1
